# PERBANDINGAN METODE KLASTER K-MEDOIDS DAN K-MEANS TERHADAP HASIL PERAMALAN KEMISKINAN DI INDONESIA MENGGUNAKAN BPNN
### by Riansyah Fazar Ramadhan

### 1. Import Dependencies

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import tensorflow as tf
from sklearn.model_selection import train_test_split

### 2. Data Preprocessing

In [2]:
data1 = pd.read_excel("E:/AKTUARIA FINAL SEASON/DATA/fpdata.xlsx", sheet_name="povt", index_col=0)
data2 = pd.read_excel("E:/AKTUARIA FINAL SEASON/DATA/fpdata.xlsx", sheet_name="unemp", index_col=0)
data3 = pd.read_excel("E:/AKTUARIA FINAL SEASON/DATA/fpdata.xlsx", sheet_name="growth", index_col=0)
data4 = pd.read_excel("E:/AKTUARIA FINAL SEASON/DATA/fpdata.xlsx", sheet_name="gini", index_col=0)

data1.drop(columns=["Sep23"], inplace=True)
data2.drop(columns=["Agu23"], inplace=True)
data3.drop(columns=["Agu23"], inplace=True)
data4.drop(columns=["Sep23"], inplace=True)

data1 = data1.transpose()
data2 = data2.transpose()
data3 = data3.transpose()
data4 = data4.transpose()

data2 = data2.values.flatten()
data3 = data3.values.flatten()
data4 = data4.values.flatten()

y_sets = data1.values.flatten()
xdf = pd.DataFrame({
        'unemp': data2,
        'growth': data3,
        'gini': data4
        })
ydf= pd.DataFrame({'pov': y_sets})
x_sets = np.array(xdf)

### 3. Descriptive Statistic

In [7]:
x_stats = xdf.describe()
y_stats = ydf.describe()
print(x_stats)
print(y_stats)

            unemp      growth        gini
count  578.000000  578.000000  578.000000
mean     5.055052    4.499152    0.353097
std      1.805693    4.086967    0.039766
min      0.880000  -21.300000    0.236000
25%      3.720000    3.321250    0.325000
50%      4.680000    5.075000    0.351000
75%      6.170000    5.773750    0.382000
max     10.950000   25.860000    0.459000
              pov
count  578.000000
mean    10.814758
std      5.638554
min      3.420000
25%      6.387500
50%      9.090000
75%     13.845000
max     28.540000


### 4. Data Preprocessing (Min-Max Normalization)

In [10]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
y_norm = scaler.fit_transform(ydf)
x_norm = scaler.fit_transform(xdf)